In [1]:
import pandas as pd

In [2]:
base = pd.read_csv('2024-divvy-tripdata.csv')
feriados = pd.read_excel('feriados_2024.xlsx')

In [3]:
base.shape

(5860568, 13)

In [4]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5860568 entries, 0 to 5860567
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 581.3+ MB


In [5]:
base.isnull().sum()

ride_id                     0
rideable_type               0
started_at                  0
ended_at                    0
start_station_name    1073951
start_station_id      1073951
end_station_name      1104653
end_station_id        1104653
start_lat                   0
start_lng                   0
end_lat                  7232
end_lng                  7232
member_casual               0
dtype: int64

In [6]:
base[base.duplicated(subset='ride_id')]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
1697610,3B5CE4D8B3EE6ED8,electric_bike,2024-05-31 23:50:04.153,2024-06-01 00:06:08.273,NaN,NaN,NaN,NaN,41.980000,-87.670000,41.980000,-87.670000,casual
1697634,60B4DDFF369931B2,electric_bike,2024-05-31 23:51:07.398,2024-06-01 00:25:37.453,NaN,NaN,NaN,NaN,41.880000,-87.650000,41.970000,-87.680000,casual
1697659,1D8856396862BE62,electric_bike,2024-05-31 23:56:12.241,2024-06-01 00:10:02.667,California Ave & Milwaukee Ave,13084,NaN,NaN,41.922714,-87.697154,41.930000,-87.720000,casual
1698918,B1B9E66D2E7C383D,classic_bike,2024-05-31 11:50:26.806,2024-06-01 09:52:57.046,Austin Blvd & Madison St,16918,Austin Blvd & Madison St,16918,41.880281,-87.774453,41.880281,-87.774453,casual
1702774,9FF6FFB668739B47,electric_bike,2024-05-31 23:53:26.419,2024-06-01 00:04:05.504,Halsted St & Wrightwood Ave,TA1309000061,Orleans St & Chestnut St (NEXT Apts),620,41.929208,-87.649222,41.898203,-87.637536,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2397298,2C2CD7C3DCBC3C39,classic_bike,2024-05-31 01:09:43.561,2024-06-01 02:09:36.476,Michigan Ave & 8th St,623,NaN,NaN,41.872773,-87.623981,NaN,NaN,casual
2399469,532780A49B9089CD,electric_bike,2024-05-31 23:57:45.184,2024-06-01 00:13:39.363,Public Rack - Garfield Park Conservatory,1059,NaN,NaN,41.886529,-87.716723,41.890000,-87.770000,casual
2400500,9B43F7CB54CFB914,electric_bike,2024-05-31 23:34:26.101,2024-06-01 00:11:15.263,Loomis St & Lexington St,13332,NaN,NaN,41.872167,-87.661361,41.870000,-87.660000,member
2401823,7817F106775A6E24,classic_bike,2024-05-31 18:01:22.459,2024-06-01 19:01:14.075,Wells St & Concord Ln,TA1308000050,NaN,NaN,41.912133,-87.634656,NaN,NaN,casual


In [7]:
base = base.drop_duplicates(subset='ride_id', keep='first')
base = base.dropna(axis=0)

In [8]:
base.shape

(4208188, 13)

In [9]:
base['started_at'] = pd.to_datetime(base['started_at'], format='mixed', errors='coerce')
base['ended_at'] = pd.to_datetime(base['ended_at'], format='mixed', errors='coerce')

base['started_at'] = base['started_at'].dt.round('s')
base['ended_at'] = base['ended_at'].dt.round('s')

In [10]:
base[base['started_at'] >= base['ended_at']].shape[0]

252

In [11]:
base[base['started_at'] < base['ended_at']].shape[0]

4207936

In [12]:
base = base[base['started_at'] < base['ended_at']]

In [13]:
base['ride_length'] = base['ended_at'] - base['started_at']

In [14]:
base['day_of_week'] = base['started_at'].dt.weekday + 1

In [15]:
mapa_dias = {
    1: 'Sunday', 2: 'Monday', 3: 'Tuesday',
    4: 'Wednesday', 5: 'Thursday', 6: 'Friday', 7: 'Saturday'
}
base['day_of_week'] = base['day_of_week'].map(mapa_dias)

In [16]:
base['month'] = base['started_at'].dt.strftime('%B')
display(base)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,month
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.903267,-87.634737,41.889177,-87.638506,member,0 days 00:07:32,Thursday,January
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902937,-87.634440,41.889177,-87.638506,member,0 days 00:07:13,Sunday,January
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902951,-87.634470,41.889177,-87.638506,member,0 days 00:08:00,Friday,January
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,TA1305000030,Larrabee St & Webster Ave,13193,41.884295,-87.633963,41.921822,-87.644140,member,0 days 00:29:49,Sunday,January
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,13253,Kingsbury St & Kinzie St,KA1503000043,41.948797,-87.675278,41.889177,-87.638506,member,0 days 00:26:12,Tuesday,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860559,36DAF3C93190E07F,classic_bike,2024-12-13 15:40:06,2024-12-13 15:46:30,Albany Ave & Bloomingdale Ave,15655,California Ave & Milwaukee Ave,13084,41.914027,-87.705126,41.922695,-87.697153,member,0 days 00:06:24,Thursday,December
5860561,71F02C3CF79B8090,classic_bike,2024-12-17 08:09:13,2024-12-17 08:15:50,Albany Ave & Bloomingdale Ave,15655,California Ave & Milwaukee Ave,13084,41.914027,-87.705126,41.922695,-87.697153,member,0 days 00:06:37,Monday,December
5860562,85AE8840FA0E4EAB,classic_bike,2024-12-18 08:22:41,2024-12-18 08:29:25,Albany Ave & Bloomingdale Ave,15655,California Ave & Milwaukee Ave,13084,41.914027,-87.705126,41.922695,-87.697153,member,0 days 00:06:44,Tuesday,December
5860565,15602635C5DF484E,electric_bike,2024-12-31 17:10:03,2024-12-31 17:17:22,Albany Ave & Bloomingdale Ave,15655,California Ave & Milwaukee Ave,13084,41.914027,-87.705126,41.922695,-87.697153,member,0 days 00:07:19,Monday,December


In [17]:
feriados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Data     19 non-null     datetime64[ns]
 1   Feriado  19 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 436.0+ bytes


In [18]:
feriados.head()

,Data,Feriado
0,2024-01-01,Ano Novo
1,2024-01-15,Dia de Martin Luther King Jr.
2,2024-02-12,Aniversário de Lincoln
3,2024-02-19,Dia dos Presidentes
4,2024-03-17,Dia de São Patrício


In [19]:
base['data_viagem'] = base['started_at'].dt.date
feriados['Data'] = pd.to_datetime(feriados['Data']).dt.date
base['is_holiday'] = base['data_viagem'].isin(feriados['Data'])
base.head(3)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,month,data_viagem,is_holiday
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.903267,-87.634737,41.889177,-87.638506,member,0 days 00:07:32,Thursday,January,2024-01-12,False
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902937,-87.634440,41.889177,-87.638506,member,0 days 00:07:13,Sunday,January,2024-01-08,False
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902951,-87.634470,41.889177,-87.638506,member,0 days 00:08:00,Friday,January,2024-01-27,False


In [20]:
base.drop(columns=['data_viagem'], inplace=True)

In [21]:
def get_season(month):
    if month in ['December', 'January', 'February']:
        return 'Inverno'
    elif month in ['March', 'April', 'May']:
        return 'Primavera'
    elif month in ['June', 'July', 'August']:
        return 'Verão'
    else:
        return 'Outono' # Setembro, Outubro, Novembro

base['seasons'] = base['month'].apply(get_season)

In [22]:
display(base)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,month,is_holiday,seasons
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.903267,-87.634737,41.889177,-87.638506,member,0 days 00:07:32,Thursday,January,False,Inverno
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902937,-87.634440,41.889177,-87.638506,member,0 days 00:07:13,Sunday,January,False,Inverno
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902951,-87.634470,41.889177,-87.638506,member,0 days 00:08:00,Friday,January,False,Inverno
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,TA1305000030,Larrabee St & Webster Ave,13193,41.884295,-87.633963,41.921822,-87.644140,member,0 days 00:29:49,Sunday,January,False,Inverno
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,13253,Kingsbury St & Kinzie St,KA1503000043,41.948797,-87.675278,41.889177,-87.638506,member,0 days 00:26:12,Tuesday,January,False,Inverno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860559,36DAF3C93190E07F,classic_bike,2024-12-13 15:40:06,2024-12-13 15:46:30,Albany Ave & Bloomingdale Ave,15655,California Ave & Milwaukee Ave,13084,41.914027,-87.705126,41.922695,-87.697153,member,0 days 00:06:24,Thursday,December,False,Inverno
5860561,71F02C3CF79B8090,classic_bike,2024-12-17 08:09:13,2024-12-17 08:15:50,Albany Ave & Bloomingdale Ave,15655,California Ave & Milwaukee Ave,13084,41.914027,-87.705126,41.922695,-87.697153,member,0 days 00:06:37,Monday,December,False,Inverno
5860562,85AE8840FA0E4EAB,classic_bike,2024-12-18 08:22:41,2024-12-18 08:29:25,Albany Ave & Bloomingdale Ave,15655,California Ave & Milwaukee Ave,13084,41.914027,-87.705126,41.922695,-87.697153,member,0 days 00:06:44,Tuesday,December,False,Inverno
5860565,15602635C5DF484E,electric_bike,2024-12-31 17:10:03,2024-12-31 17:17:22,Albany Ave & Bloomingdale Ave,15655,California Ave & Milwaukee Ave,13084,41.914027,-87.705126,41.922695,-87.697153,member,0 days 00:07:19,Monday,December,False,Inverno


In [23]:
base.nunique()

ride_id               4207936
rideable_type               3
started_at            3694508
ended_at              3705289
start_station_name       1789
start_station_id         1744
end_station_name         1799
end_station_id           1750
start_lat              429859
start_lng              416551
end_lat                  2332
end_lng                  2333
member_casual               2
ride_length             20051
day_of_week                 7
month                      12
is_holiday                  2
seasons                     4
dtype: int64

In [24]:
base.shape

(4207936, 18)

In [25]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4207936 entries, 0 to 5860566
Data columns (total 18 columns):
 #   Column              Dtype          
---  ------              -----          
 0   ride_id             object         
 1   rideable_type       object         
 2   started_at          datetime64[ns] 
 3   ended_at            datetime64[ns] 
 4   start_station_name  object         
 5   start_station_id    object         
 6   end_station_name    object         
 7   end_station_id      object         
 8   start_lat           float64        
 9   start_lng           float64        
 10  end_lat             float64        
 11  end_lng             float64        
 12  member_casual       object         
 13  ride_length         timedelta64[ns]
 14  day_of_week         object         
 15  month               object         
 16  is_holiday          bool           
 17  seasons             object         
dtypes: bool(1), datetime64[ns](2), float64(4), object(10), timedelta64[

In [26]:
base.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
ride_length           0
day_of_week           0
month                 0
is_holiday            0
seasons               0
dtype: int64

In [28]:
base.to_csv('tripdata.csv', index=False)